# Complex body TFA Inversion

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA) data on a regular grid of a model with a complex geometry.

In [81]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os

from fatiando import utils
from fatiando.gravmag import polyprism
from fatiando.mesher import PolygonalPrism
from fatiando.vis import mpl, myv
from matplotlib import colors, ticker, cm
from IPython.display import Image as img
from matplotlib.mlab import normpdf

### Auxiliary functions

In [82]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing model parameters

In [83]:
model = 'complex_data/complex_model.pickle'
data = 'complex_data/flightlines_grid_1300pts_100k/data.pickle'

In [84]:
with open(model) as w:
        complex_model = pickle.load(w)

### Observed data

In [85]:
with open(data) as w:
        complex_data = pickle.load(w)

In [86]:
# observed data and observation points
dobs = complex_data['tfa_obs']
xp = complex_data['grid'][0]
yp = complex_data['grid'][1]
zp = complex_data['grid'][2]
N = complex_data['grid'][3]

### Parameters of the initial model

In [87]:
M = 15 # number of vertices per prism
L = 8 # number of prisms
P = L*(M+2) + 1 # number of parameters

# magnetization direction
# incs = complex_model['inc']
# decs = complex_model['dec']
#intensity = complex_model['intensity']
incs = 45.
decs = -10
intensity = 10.

# depth to the top, thickness and radius
z0 = complex_model['z0']
dz = 600.
r = 500.

# total magnetization
props = {'magnetization': utils.ang2vec(
        intensity, incs, decs)}
rin = np.zeros(M) + r
m0 = np.hstack((rin, np.zeros(2)))
m0 = np.resize(m0, P - 1) 
m0 = np.hstack((m0, dz)) # inicial parameters vector
model0 = mfun.param2polyprism(m0, M, L, z0, props) # list of classes of prisms

# main field
inc, dec = complex_data['main_field']

# predict data
d0 = polyprism.tf(xp, yp, zp, model0, inc, dec)

### Limits

In [88]:
# limits for parameters in meters
rmin = 100.
rmax = 2000.
x0min = -1500.
x0max = 1500.
y0min = -1500.
y0max = 1500.
dzmin = 200.
dzmax = 700.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Variation

In [89]:
# variation for derivatives
deltax = 0.01*np.max(x0max)
deltay = 0.01*np.max(y0max)
deltar = 0.01*np.max(rmax)
deltaz = 0.01*np.max(dzmax)

### Outcropping parameters

In [90]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = complex_model['param_vec'][:M+2]

### Folder to save the results

In [91]:
if complex_data['grid'][4] == 'flightlines':
    if incs == complex_model['inc'] and decs == complex_model['dec']:
        mypath = 'tfa_inversion/true_direction_flightlines/r%d_int%d_z0%d_dz%d' % (rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
    else:
        mypath = 'tfa_inversion/wrong_direction_flightlines/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
else:
    if incs == complex_model['inc'] and decs == complex_model['dec']:
        mypath = 'tfa_inversion/true_direction_regular/r%d_int%d_z0%d_dz%d' % (rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
    else:
        mypath = 'tfa_inversion/wrong_direction_regular/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)

In [92]:
# output of inversion
complex_inversion = dict()

### Regularization parameters

In [93]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 1000.00
dlamb = 10.      # step for Marquadt's parameter

a1 = 0.0001  # adjacent radial distances within each prism
a2 = 0.0001   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 0.001     # vertically adjacent origins
a6 = 0.0000001   # zero order Tikhonov on adjacent radial distances
a7 = 0.0000001     # zero order Tikhonov on thickness of each prism

In [94]:
delta = np.array([deltax, deltay, deltar, deltaz])
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [95]:
itmax = 30
itmax_marq = 10
tol = 1.0e-5     # stop criterion

In [96]:
complex_inversion['x'] = xp
complex_inversion['y'] = yp
complex_inversion['z'] = zp
complex_inversion['observed_data'] = dobs

In [97]:
complex_inversion['inc_dec'] = [incs, decs]
complex_inversion['z0'] = z0
complex_inversion['initial_dz'] = dz
complex_inversion['intial_r'] = r
complex_inversion['initial_estimate'] = model0
complex_inversion['initial_data'] = d0
complex_inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
complex_inversion['regularization'] = alpha
complex_inversion['tol'] = tol
complex_inversion['main_field'] = [complex_data['main_field'][0], complex_data['main_field'][1]]

### Inversion

In [98]:
d_fit, m_est, model_est, phi_list = mfun.levmarq_tf(
    xp, yp, zp, m0, M, L, delta,
    itmax, itmax_marq, lamb,
    dlamb, tol, mmin, mmax,
    m_out, dobs, inc, dec,
    props, alpha, z0, dz
)

it:  0   it_marq:  0   lambda: 1e+03   misfit: 1.80974e+04
it:  1   it_marq:  0   lambda: 1e+02   misfit: 1.02696e+04
it:  2   it_marq:  0   lambda: 1e+01   misfit: 4.06179e+03
it:  3   it_marq:  0   lambda: 1e+00   misfit: 1.60924e+03
it:  4   it_marq:  0   lambda: 1e-01   misfit: 5.57715e+02
it:  5   it_marq:  0   lambda: 1e-02   misfit: 9.47716e+02
it:  5   it_marq:  1   lambda: 1e-01   misfit: 2.66903e+02
it:  6   it_marq:  0   lambda: 1e-02   misfit: 1.24160e+03
it:  6   it_marq:  1   lambda: 1e-01   misfit: 3.17498e+02
it:  6   it_marq:  2   lambda: 1e+00   misfit: 8.22517e+02
it:  6   it_marq:  3   lambda: 1e+01   misfit: 1.12120e+03
it:  6   it_marq:  4   lambda: 1e+02   misfit: 4.97622e+02
it:  6   it_marq:  5   lambda: 1e+03   misfit: 2.61155e+02
it:  7   it_marq:  0   lambda: 1e+02   misfit: 2.27210e+02
it:  8   it_marq:  0   lambda: 1e+01   misfit: 1.62047e+02
it:  9   it_marq:  0   lambda: 1e+00   misfit: 1.20489e+02
it: 10   it_marq:  0   lambda: 1e-01   misfit: 3.87195e+

# Results

In [99]:
complex_inversion['data_fit'] = d_fit
complex_inversion['estimate'] = m_est
complex_inversion['prisms'] = model_est
complex_inversion['objective'] = phi_list
complex_inversion['residual'] = dobs - d_fit

In [100]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(complex_inversion, f)